## Setup Tools

In [216]:
import sys
from time import time
import urllib2

# Xor Two Str
def strxor(s1, s2):
    return "".join([ chr(ord(c2) ^ ord(c1)) for (c1, c2) in zip(s1, s2)])

# Replace context in str with substr at pos
def strrep(str, substr, pos):
    endpos = pos + len(substr)
    return str[:pos] + substr + str[endpos:]

## Given Code

In [217]:
TARGET = 'http://crypto-class.appspot.com/po?er='
#--------------------------------------------------------------
# padding oracle
#--------------------------------------------------------------
class PaddingOracle(object):
    def query(self, q):
        target = TARGET + urllib2.quote(q)    # Create query URL
        req = urllib2.Request(target)         # Send HTTP request to server
        try:
            f = urllib2.urlopen(req)          # Wait for response
        except urllib2.HTTPError, e:
            return e.code
        return 200

## Try Different Query

In [218]:
def try_byte(query, byte_at, pad,lens):
    po = PaddingOracle()

    def try_bit(b):
        q = strrep(query, chr(ord(query[byte_at]) ^ pad ^ b), byte_at)
        #print q.encode('hex')
        
        http_status = po.query(q.encode('hex'))
        if http_status in (404,200):
            # Padding Found
            return b

        else :
            # Not Found
            return 0

    if try_bit(0x09): return 0x09
    if try_bit(0x20): return 0x20

    for i in xrange(0x7f, 0x00, -1):
        print ('\rCurrent [%.2f%%] / Total [%.2f%%]' % ( (float(0x7f-i) / int(0xff))*100 , ( pad / float(lens))*100 ) ),
        sys.stdout.flush()
        if try_bit(i): 
            return i

    for i in xrange(0x80, 0xFF):
        print ('\rCurrent Progress [%f]' % (float(i-0x80) + int(0x7f) / int(0xff)) ),
        sys.stdout.flush()
        if try_bit(i): 
            return i
 
    return None

def padding_attack(query, start, end):
    guess = ''
    for i in xrange(end - 1 , start - 1, -1):   
        padlen = end - i
        subst = strxor(strxor(query[i+1:end], guess), chr(padlen) * padlen)
        q = strrep(query, subst, i+1)
        
        g = try_byte(q, i, padlen,end-start)
        if g is None:
            print 'No Byte Found at %d' %i
            return None
        
        guess = chr(g) + guess
    
    return guess

## Main

In [219]:
start_query='f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c0bdf302936266926ff37dbf7035d5eeb4'.decode('hex')

begin = time()

print("\nBlock 2")
s3 = padding_attack(start_query, 32, 48)

print("\nBlock 1")
s2 = padding_attack(start_query[:48], 16, 32)

print("\nBlock 0")
s1 = padding_attack(start_query[:32], 0, 16)

end = time()

print '\n' + s1 + s2 + s3
print ("\nFinished in %ds" % (end-begin) )


Block 2
Current [4.71%] / Total [100.00%]                                                                                                                                                              
Block 1
Current [11.76%] / Total [100.00%]                                                                                                                                                                                                                                                                                                                                                   
Block 0
Current [16.86%] / Total [100.00%]                                                                                                                                                                                                                                                                                                                                                                             
The Magic